In [2]:
#!pip install datapackage --upgrade
#!pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 85 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 72 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 68 kB 8.3 MB/s eta 0:00:011
     |████████████████████████████████| 41 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 8.7 MB 2.7 MB/s eta 0:00:01     |██▌                             | 665 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 2.2 MB/s eta 0:00:01
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7399 sha256=e59c682bf184ed2baca4e79869e794ed60afd346ddc341432c1b1e7e18d349af
  Stored in directory: /Users/melissayu/Library/Caches/pip/wheels/b5/eb/b6/e409f80d7fec532e4240dda9562cad374257d2dd010b40cfff
Successfully built linear-tsv


# This file is for data preparation. It is not necessary to run, since the files we used for analysis are already exported.
The reason is that a free API key can only get 100 companies's news, so I tried 6 different api keys. 
However, If running, it requires a lot of time and for every 100 companies you need to change a api key, 
which I listed below.

In [1]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/core/s-and-p-500-companies/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        data = pd.read_csv(resource.descriptor['path'])
        print (data)

    Symbol                 Name                  Sector
0      MMM                   3M             Industrials
1      AOS          A. O. Smith             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV               AbbVie             Health Care
4     ABMD              Abiomed             Health Care
..     ...                  ...                     ...
500    YUM          Yum! Brands  Consumer Discretionary
501   ZBRA   Zebra Technologies  Information Technology
502    ZBH        Zimmer Biomet             Health Care
503   ZION        Zions Bancorp              Financials
504    ZTS               Zoetis             Health Care

[505 rows x 3 columns]
    Symbol                 Name                  Sector
0      MMM                   3M             Industrials
1      AOS          A. O. Smith             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV               AbbVie             Health Care
4     ABMD              

In [3]:
company_list = list(data.Name)
symbol_list = list(data.Symbol)

In [5]:
import yfinance as yf
import pandas as pd

stock = pd.DataFrame()

for name in symbol_list:
    ticker_object = yf.Ticker(name)
    ticker_object_historical = ticker_object.history(start="2022-03-20", end="2022-04-20")
    ticker_object_historical['Symbol'] = name
    stock = stock.append(ticker_object_historical)

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- INFO: No data found, symbol may be delisted
- KSU: No data found, symbol may be delisted
- VIAC: No data found, symbol may be delisted
- WLTW: No data found, symbol may be delisted
- XLNX: No data found, symbol may be delisted


In [ ]:
#6 month data for future use
#for name in symbol_list:
    #ticker_object = yf.Ticker(name)
    #ticker_object_historical = ticker_object.history(start="2021-10-20", end="2022-04-20")
    #ticker_object_historical['Symbol'] = name
    #stock = stock.append(ticker_object_historical)

The output generated above shows that the data of six companies were not found:

- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- INFO: No data found for this date range, symbol may be delisted
- KSU: No data found, symbol may be delisted
- WLTW: No data found, symbol may be delisted
- XLNX: No data found, symbol may be delisted

So we manual insert these six companies into our database
The reason that it doesn't insert is that the symbol is different
Symbol in yfinance referring to the first two companied should be converted from ("BRK.B", "BF.B") to ("BRK-B", "BF-B")
Yahoo Finance official website do not provide the stock price data of the other four companies in the last month, 
so we excluded these four companies

In [6]:
stock1 = pd.DataFrame(stock)

In [7]:
#1 month for our demo
delisted_symbol = ("BRK-B", "BF-B")
stock2 = pd.DataFrame()
for ticker in delisted_symbol:
    ticker_object = yf.Ticker(ticker)
    ticker_object_historical = ticker_object.history(start="2022-03-20", end="2022-04-20")
    ticker_object_historical['Symbol'] = ticker
    stock2 = stock2.append(ticker_object_historical)
stock1 = stock1.append(stock2)

In [ ]:
#6 months maybe in the future
#delisted_symbol = ("BRK-B", "BF-B")
#stock2 = pd.DataFrame()
#for ticker in delisted_symbol:
    #ticker_object = yf.Ticker(ticker)
    #ticker_object_historical = ticker_object.history(start="2021-10-20", end="2022-04-20")
    #ticker_object_historical['Symbol'] = ticker
    #stock2 = stock2.append(ticker_object_historical)
#stock1 = stock1.append(stock2)

In [8]:
stock1

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Adj Close
Date,,,,,,,,,
2022-03-21,148.690002,149.740005,147.399994,148.580002,4514900.0,0.0,0.0,MMM,NaN
2022-03-22,149.449997,150.839996,148.500000,149.940002,4252000.0,0.0,0.0,MMM,NaN
2022-03-23,149.690002,149.779999,147.559998,147.679993,2637200.0,0.0,0.0,MMM,NaN
2022-03-24,148.119995,149.229996,147.399994,148.979996,2028100.0,0.0,0.0,MMM,NaN
2022-03-25,149.270004,151.179993,148.910004,150.460007,2247800.0,0.0,0.0,MMM,NaN
...,...,...,...,...,...,...,...,...,...
2022-04-12,68.269997,68.970001,67.879997,68.239998,1196200.0,0.0,0.0,BF-B,NaN
2022-04-13,68.010002,69.089996,67.550003,68.919998,1136300.0,0.0,0.0,BF-B,NaN
2022-04-14,69.239998,69.800003,68.290001,68.500000,4782200.0,0.0,0.0,BF-B,NaN


In [9]:
stock1 = stock1.reset_index()
stock1 = stock1.filter(items=['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [10]:
stock1

,Symbol,Date,Open,High,Low,Close,Volume
0,MMM,2022-03-21,148.690002,149.740005,147.399994,148.580002,4514900.0
1,MMM,2022-03-22,149.449997,150.839996,148.500000,149.940002,4252000.0
2,MMM,2022-03-23,149.690002,149.779999,147.559998,147.679993,2637200.0
3,MMM,2022-03-24,148.119995,149.229996,147.399994,148.979996,2028100.0
4,MMM,2022-03-25,149.270004,151.179993,148.910004,150.460007,2247800.0
...,...,...,...,...,...,...,...
10482,BF-B,2022-04-12,68.269997,68.970001,67.879997,68.239998,1196200.0
10483,BF-B,2022-04-13,68.010002,69.089996,67.550003,68.919998,1136300.0
10484,BF-B,2022-04-14,69.239998,69.800003,68.290001,68.500000,4782200.0
10485,BF-B,2022-04-18,68.349998,68.699997,67.160004,67.459999,1334500.0


In [11]:
stock1.to_csv('stock_price.csv',index=False)

In [ ]:
#6months
#stock1.to_csv('stock_price_6_month.csv',index=False)

# We have tried 6 different api keys because one free api key can only extract 100 companies, so you can replace the following apiKey in the area of "apiKey=".
'52130cd4c70747e3baaa9b6068ccf32f'
'bbb9fbdfe2684646a44b963c18a0a1c4'
'eafdaf97cf464efaa9025bb8426d01d6'
'60d2e3e6d29846818a60c9311bea67bc'
'20d1e88062794c63a20b9d76f23c9d16'
'2c95544c1f62467dac17e3dd1988e848'

In [279]:
def get_company_news(company_name):
 
    try:
        # Get store review count
        response = requests.request('GET', 'https://newsapi.org/v2/everything?'
       'q=%s&'
       'from=2022-03-22&'
       'sortBy=popularity&'
       'apiKey=20d1e88062794c63a20b9d76f23c9d16' % (company_name,))
        data = response.json()
        for i in data["articles"][:20]:
            i["company"] = company_name
        
        return data["articles"][:20]
    
    except:
        dic = {}
        list1 = []
        dic["company"] = company_name
        dic["description"] = "No data found"
        list1.append(dic)
        
        return list1
    
#url = ('https://newsapi.org/v2/everything?q=apple&from=2022-03-22&sortBy=popularity&apiKey=f45df9d527394ef7bb01677f475c7951')

In [239]:
#First 100 companies
news = []
for company in company_list[0:100]:
    news = news + get_company_news(company)

In [252]:
#second 100 companies
news1 = []
for company in company_list[100:200]:
    news1 = news1 + get_company_news(company)

In [266]:
#third 100 companies
news2 = []
for company in company_list[200:300]:
    news2 = news2 + get_company_news(company)

In [270]:
#fourth 100 companies
news3 = []
for company in company_list[300:400]:
    news3 = news3 + get_company_news(company)

In [276]:
#fifth 100 companies
news4 = []
for company in company_list[400:500]:
    news4 = news4 + get_company_news(company)

In [280]:
#last few companies
news5 = []
for company in company_list[500:]:
    news5 = news5 + get_company_news(company)

In [240]:
len(news)

979

In [247]:
len(news1)

1000

In [253]:
len(news2)

991

In [258]:
len(news3)

991

In [267]:
len(news4)

1944

In [271]:
len(news5)

1987

In [2]:
news_data = news + news1 + news2 + news3 + news4 + news5 

NameError: name 'news' is not defined

In [1]:
len(news_data)

NameError: name 'news_data' is not defined

In [290]:
records = json.dumps(news_data, indent = 4)

In [291]:
with open("news.json", "w") as outfile:
    outfile.write(records)

In [4]:
import json
with open('news.json') as f:
    news = json.load(f)
print(len(news))

9950


In [323]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import connections
from elasticsearch.helpers import bulk
from elasticsearch_dsl import Search
from elasticsearch_dsl import Q
from pprint import pprint
from datetime import datetime

In [324]:
client = Elasticsearch(hosts=["localhost"],
                     port=9200, 
                     http_auth=('elastic', '123456'), 
                     ca_certs='http_ca.crt',
                     use_ssl=True, 
                     verify_certs=True)

In [325]:
from elasticsearch.helpers import bulk
resp = bulk(client, news, index = 'news')

In [327]:
s = Search(using=client, index="news") \
        .query("match", company="Apple")

el_response = s.execute() # by default you get 10 hits

for hit in el_response['hits']['hits']:
    pprint(hit['_source'].to_dict())

{'author': 'Khamosh Pathak',
 'company': 'Apple',
 'content': 'HEIC is a solid format for Apple devices. It includes Live '
            'Photos, Apples post-processing, and it still manages to weigh way '
            'below what a JPG would. While it works fine if youre sharing a '
            'photo among… [+2667 chars]',
 'description': 'HEIC is a solid format for Apple devices. It includes Live '
                'Photos, Apple’s post-processing, and it still manages to '
                'weigh way below what a JPG would. While it works fine if '
                'you’re sharing a photo among other Apple devices, you run '
                'into a problem when you need…',
 'publishedAt': '2022-04-13T15:00:00Z',
 'source': {'id': None, 'name': 'Lifehacker.com'},
 'title': 'The Fastest Ways to Convert Your iPhone HEIC Photos to JPG (So You '
          'Can Actually Use Them)',
 'url': 'https://lifehacker.com/the-fastest-ways-to-convert-your-iphone-heic-photos-to-1848768694',
 'urlToImage